# 05: Volatility Clustering (ボラティリティ・クラスタリング)

このノートでは、ローリング標準偏差と ATR を比較し、
さらに \(|r_t|\) と \(r_t^2\) の自己相関を見てボラティリティの持続性を確認します。

数式:

$$\sigma_t^{(w)} = \mathrm{std}(r_{t-w+1},...,r_t)$$

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

SYMBOL = os.getenv("SYMBOL", "QQQ")
PERIOD = "5y"
INTERVAL = "1d"

plt.style.use("seaborn-v0_8")

In [ ]:
# --- Data preparation ---
df = yf.download(SYMBOL, period=PERIOD, interval=INTERVAL, auto_adjust=False, progress=False)
if isinstance(df.columns, pd.MultiIndex):
    high = df[("High", SYMBOL)]
    low = df[("Low", SYMBOL)]
    close = df[("Close", SYMBOL)]
else:
    high, low, close = df["High"], df["Low"], df["Close"]

data = pd.DataFrame({"high": high, "low": low, "close": close}).dropna()
data["ret"] = np.log(data["close"] / data["close"].shift(1))

# Rolling volatility
data["vol20"] = data["ret"].rolling(20).std(ddof=0)
data["vol60"] = data["ret"].rolling(60).std(ddof=0)

# ATR14
prev_close = data["close"].shift(1)
tr = pd.concat([
    (data["high"] - data["low"]).abs(),
    (data["high"] - prev_close).abs(),
    (data["low"] - prev_close).abs(),
], axis=1).max(axis=1)
data["atr14"] = tr.rolling(14).mean()
# Make ATR dimensionless by normalizing with close
data["atr14_norm"] = data["atr14"] / data["close"]

data = data.dropna()
data.tail()

## ACF を手実装して可視化

`statsmodels` なしで自己相関を計算します。

In [ ]:
def simple_acf(series: pd.Series, max_lag: int = 30) -> pd.Series:
    x = pd.Series(series).dropna().to_numpy(dtype=float)
    x = x - x.mean()
    denom = np.dot(x, x)
    if denom == 0:
        return pd.Series([np.nan] * max_lag, index=range(1, max_lag + 1))

    vals = []
    for lag in range(1, max_lag + 1):
        if lag >= len(x):
            vals.append(np.nan)
        else:
            vals.append(np.dot(x[lag:], x[:-lag]) / denom)
    return pd.Series(vals, index=range(1, max_lag + 1))

acf_abs = simple_acf(data["ret"].abs(), max_lag=40)
acf_sq = simple_acf((data["ret"] ** 2), max_lag=40)

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(13, 9), sharex=False)

axes[0].plot(data.index, data["vol20"], label="Rolling std 20")
axes[0].plot(data.index, data["vol60"], label="Rolling std 60")
axes[0].plot(data.index, data["atr14_norm"], label="ATR14 / Close", alpha=0.9)
axes[0].set_title(f"{SYMBOL}: Rolling Volatility vs ATR14")
axes[0].legend()

lags = np.arange(1, len(acf_abs) + 1)
axes[1].stem(lags - 0.1, acf_abs.values, linefmt="C0-", markerfmt="C0o", basefmt="k-", label="ACF(|ret|)")
axes[1].stem(lags + 0.1, acf_sq.values, linefmt="C1-", markerfmt="C1s", basefmt="k-", label="ACF(ret^2)")
axes[1].set_title("Autocorrelation of |ret| and ret^2")
axes[1].set_xlabel("Lag")
axes[1].legend()

plt.tight_layout()
plt.show()

## What you should notice

- 価格が荒れている局面で、`vol20` / `vol60` / `atr14_norm` が同時に上がりやすい。
- ACF(|ret|) や ACF(ret^2) は、しばしば正の値がしばらく残る。
- これは「大きく動く時期が続く」= ボラティリティ・クラスタリングの観測的証拠。
- ATR はレンジ情報を使うので、終値ベース標準偏差と補完的に使える。